In [ ]:
!pip install datasets transformers[torch] seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 33.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2

In [ ]:
from datasets import load_from_disk, DatasetDict
from sklearn.model_selection import KFold
from transformers import AutoModelForTokenClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification
import numpy as np
import datasets
import torch

In [ ]:
dataset = load_from_disk("drive/MyDrive/ner_dataset_conll_format_impression_yok")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/convbert-base-turkish-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/83.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

In [ ]:
example_text = dataset['train'][0]

tokenized_input = tokenizer(example_text["tokens"], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

word_ids = tokenized_input.word_ids()

print(word_ids)

''' As we can see, it returns a list with the same number of elements as our processed input ids, mapping special tokens to None and all other tokens to their respective word. This way, we can align the labels with the processed input ids. '''

tokenized_input

[None, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 7, 8, 9, 9, 9, 10, 11, 12, 13, 13, 13, 14, 15, 16, 17, 17, 18, 19, 20, 20, 20, 20, 21, 22, 22, 23, 23, 24, 24, 24, 25, 25, 26, 26, 26, 26, 26, 27, 28, 29, 29, 30, 31, 31, 31, 31, 32, 32, 32, 33, 34, 35, 36, 37, 37, 38, 38, 38, 38, 39, 40, 41, 42, 43, 44, 45, 46, 46, 46, 46, 47, 47, 48, 48, 49, 49, 50, 50, 51, 51, 51, 51, 52, 53, 54, 55, 55, 55, 56, 56, 56, 57, 57, 58, 59, 60, 60, 61, 61, 61, 62, 62, 62, 63, 63, 63, 64, None]


{'input_ids': [2, 10757, 3766, 2864, 3538, 28525, 1040, 30242, 22613, 1074, 10494, 27937, 4336, 6830, 30, 2776, 22169, 2008, 20016, 3481, 16, 6696, 16, 6696, 5161, 7419, 6570, 2067, 18, 8873, 13135, 2032, 7300, 1986, 16, 2267, 13135, 2719, 7300, 1986, 1992, 2267, 13102, 2135, 5689, 9969, 5220, 7085, 4082, 15450, 2433, 6915, 2016, 18, 12, 13135, 18258, 8650, 1024, 35, 13, 2874, 2550, 13135, 1988, 20471, 2235, 2462, 8038, 19943, 6915, 2016, 18, 3644, 13135, 2500, 2612, 30385, 9067, 21712, 23044, 2722, 1987, 9230, 4028, 4379, 92, 3949, 8215, 31820, 6113, 2410, 8963, 1972, 7419, 1988, 7442, 8829, 7419, 1988, 14325, 2174, 29870, 1007, 6223, 18, 23166, 16162, 5710, 2104, 22169, 2008, 14387, 18483, 11434, 3773, 4060, 20177, 25445, 7031, 2045, 29870, 2752, 18, 28820, 1128, 30, 21774, 15602, 17214, 25, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    """
    Function to tokenize and align labels with respect to the tokens. This function is specifically designed for
    Named Entity Recognition (NER) tasks where alignment of the labels is necessary after tokenization.

    Parameters:
    examples (dict): A dictionary containing the tokens and the corresponding NER tags.
                     - "tokens": list of words in a sentence.
                     - "ner_tags": list of corresponding entity tags for each word.

    label_all_tokens (bool): A flag to indicate whether all tokens should have labels.
                             If False, only the first token of a word will have a label,
                             the other tokens (subwords) corresponding to the same word will be assigned -100.

    Returns:
    tokenized_inputs (dict): A dictionary containing the tokenized inputs and the corresponding labels aligned with the tokens.
    """
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `<s>` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
q = tokenize_and_align_labels(dataset['train'][4:5])
print(q)

{'input_ids': [[2, 10757, 3766, 2864, 3538, 28525, 1040, 30242, 22613, 1074, 10494, 27937, 4336, 6830, 27510, 1990, 13642, 1072, 7479, 39, 3773, 4292, 1988, 3340, 16, 17104, 17494, 30140, 31613, 18, 8873, 13135, 1988, 2747, 30385, 4226, 1986, 13949, 15173, 1048, 5743, 3233, 2846, 6350, 4746, 18258, 8650, 1985, 26349, 2435, 26904, 22958, 7031, 14207, 14325, 2174, 1980, 8304, 18, 11146, 2037, 26379, 10657, 11144, 9676, 14325, 18073, 16, 11144, 2235, 2462, 8038, 2510, 16, 14983, 30284, 2010, 2054, 2446, 10297, 2872, 11631, 27698, 18, 8873, 14387, 18483, 2356, 4332, 2114, 10657, 6509, 1988, 3773, 4060, 4737, 1992, 7419, 1988, 24542, 25445, 7031, 2045, 29870, 2752, 18, 3644, 13135, 1988, 2747, 30385, 4226, 1986, 13949, 15173, 1048, 5743, 3233, 2846, 6350, 4746, 18258, 8650, 1985, 26349, 2435, 26904, 22958, 7031, 14207, 14325, 2174, 1980, 8304, 18, 11146, 2037, 26379, 10657, 11144, 9676, 14325, 18073, 16, 11144, 2235, 2462, 8038, 2510, 16, 14983, 30284, 11631, 27698, 18, 3220, 1059, 13135, 2

In [ ]:
for token, label in zip(tokenizer.convert_ids_to_tokens(q["input_ids"][0]),q["labels"][0]):
    print(f"{token:_<40} {label}")

[CLS]___________________________________ -100
BİL_____________________________________ 0
##AT____________________________________ 0
##ER____________________________________ 0
##AL____________________________________ 0
MA______________________________________ 0
##M_____________________________________ 0
##OG____________________________________ 0
##RAF___________________________________ 0
##İ_____________________________________ 0
İN______________________________________ 0
##CEL___________________________________ 0
##EM____________________________________ 0
##ESİ___________________________________ 0
Meme____________________________________ 1
##ler___________________________________ 1
AC______________________________________ 0
##R_____________________________________ 0
Tip_____________________________________ 2
C_______________________________________ 2
pat_____________________________________ 2
##ern___________________________________ 2
##de____________________________________ 2
olup____

In [ ]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/583 [00:00<?, ? examples/s]

Map:   0%|          | 0/195 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("dbmdz/convbert-base-turkish-cased")

Some weights of ConvBertForTokenClassification were not initialized from the model checkpoint at dbmdz/convbert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
metric = datasets.load_metric("seqeval")

<ipython-input-21-97308d58ec09>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


In [ ]:
import os
kf = KFold(n_splits=5)
results = []
best_f1 = 0
best_model = None

In [ ]:
def compute_metrics(eval_preds):
        pred_logits, labels = eval_preds
        pred_logits = np.argmax(pred_logits, axis=2)
        predictions = [
            [label_list[pred] for pred, label in zip(prediction, label) if label != -100]
            for prediction, label in zip(pred_logits, labels)
        ]
        true_labels = [
            [label_list[label] for pred, label in zip(prediction, label) if label != -100]
            for prediction, label in zip(pred_logits, labels)
        ]
        results = metric.compute(predictions=predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [ ]:
for fold, (train_index, val_index) in enumerate(kf.split(tokenized_datasets["train"])):
    print(f"Fold {fold + 1}")

    train_dataset = tokenized_datasets["train"].select(train_index)
    val_dataset = tokenized_datasets["train"].select(val_index)
    test_dataset = tokenized_datasets["test"]  # Assuming you have a separate test set

    args = TrainingArguments(
        f"test-ner-fold-{fold}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        save_total_limit=1,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
    )

    data_collator = DataCollatorForTokenClassification(tokenizer)

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=data_collator,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    try:
        trainer.train()
        eval_result = trainer.evaluate(eval_dataset=val_dataset)
        results.append(eval_result)

        if eval_result["eval_f1"] > best_f1:
            best_f1 = eval_result["eval_f1"]
            best_model = model
            trainer.save_model("best_model")

        test_result = trainer.evaluate(eval_dataset=test_dataset)
        print(f"Test results for fold {fold}: {test_result}")
    except RuntimeError as e:
        print(f"Runtime error in fold {fold}: {e}")

avg_results = {k: np.mean([result[k] for result in results]) for k in results[0].keys()}
print(f"Average results across all folds: {avg_results}")

Fold 1


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
